Problem Set 2
Maire Taegan Mullane
4/13/25

In [47]:
import pandas as pd
import numpy as np
import math
import scipy as sp
import matplotlib.pyplot as plt
from itertools import combinations

In [4]:
file1_path = "census.mat"
#read .mat file without a dictionary into dataframe
# Note: The .mat file should be in the same directory as this script

mat_data = sp.io.loadmat(file1_path)

In [7]:
# Extract the variables
educ = mat_data['educ'].flatten()
y = mat_data['y'].flatten()
yob = mat_data['yob'].flatten()

# Create a DataFrame
df = pd.DataFrame({
    'educ': educ,
    'y': y,
    'yob': yob
})

df

,educ,y,yob
0,0,5.790019,30
1,0,5.952494,30
2,0,5.315949,30
3,0,5.595926,30
4,0,6.068915,30
...,...,...,...
329504,1,6.323980,39
329505,1,5.847161,39
329506,0,5.909597,39
329507,0,6.047781,39


1.a.

In [13]:
Y_1 = df[df.educ== 1]['y'].mean()
Y_0 = df[df.educ== 0]['y'].mean()

diff = (Y_1- Y_0)
print(f"Difference in means is: {diff:.2f}")


Difference in means is: 0.38


In [15]:
N = float(len(df))
M = float(len(df[df.educ == 1]))

# Calculate the variance of the difference in means
se_0 =(1/(N-M-1)) * sum((df[df.educ == 0].y - Y_0)**2)
se_1 = 1/(M - 1) * sum((df[df.educ == 1].y - Y_1)**2)
var = se_0/(N-M) + se_1/M
print(f"Variance of the difference in means is: {var:.8f}")

Variance of the difference in means is: 0.00000534


1.b.

In [17]:
def bootstrap_diff(df, n=None):
    df_b = df.sample(n, replace=True)
    
    Y_1_b = df_b[df_b.educ == 1]['y'].mean()
    Y_0_b = df_b[df_b.educ == 0]['y'].mean()
    return Y_1_b - Y_0_b

def bootstrap_se(df, B=1000):
    n = len(df)
    diffs = np.zeros(B)
    for i in range(B):
        diffs[i] = bootstrap_diff(df, n)
    return np.std(diffs)

# Set the number of bootstrap samples
B = 1000
# Calculate the bootstrap standard error
bootstrap_se_value = bootstrap_se(df, B=B)
print(f"Bootstrap standard error of the difference in means is: {bootstrap_se_value:.8f}")


Bootstrap standard error of the difference in means is: 0.00226623


1.c.

In [19]:
yob_values = df['yob'].unique()
np.zeros(len(yob_values))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
def cluster_diff(df, i=None):
    df_c = df[df.yob == i]
    
    Y_1_b = df_c[df_c.educ == 1]['y'].mean()
    Y_0_b = df_c[df_c.educ == 0]['y'].mean()
    return Y_1_b - Y_0_b

def cluster_se(df, cluster = None):
    diffs = np.zeros(len(cluster))
    for i in range(len(cluster)):
        diffs[i] = cluster_diff(df, cluster[i])
    return np.mean(diffs)

# Calculate the LZ standard error
lz_se = cluster_se(df, cluster = yob_values)
print(f"LZ standard error of the difference in means is: {lz_se:.8f}")


LZ standard error of the difference in means is: 0.37873455


1.d.

In [25]:
w_k = np.zeros(len(yob_values))
for i in range(len(yob_values)):
    w_k[i] = np.mean(df[df['yob']==yob_values[i]].educ)
w_k

array([0.36417475, 0.38279436, 0.38831455, 0.39406849, 0.39660358,
       0.40316724, 0.40537397, 0.40713592, 0.41498453, 0.41974585])

In [42]:
np.random.choice(w_k, size=1, replace=True)[0]*len(df)

np.float64(138310.03382209188)

In [45]:
def cluster_diff(df, i=None):
    df_c = df[df.yob == i]
    df_c_1 = df_c[df_c.educ == 1]
    df_c_0 = df_c[df_c.educ == 0]

    w_kg = np.random.choice(w_k, size=1, replace=True)[0]
    n_c = len(df_c)
    df_cb_1 = df_c_1.sample(int(n_c*w_kg), replace=True)
    df_cb_0 = df_c_0.sample(int(n_c*(1-w_kg)), replace=True)

    Y_1_b = df_cb_1['y'].mean()
    Y_0_b = df_cb_0['y'].mean()
    
    return Y_1_b - Y_0_b

def cluster_se(df, cluster = None):
    diffs_c = np.zeros(len(cluster))
    for i in range(len(cluster)):
        diffs_c[i] = cluster_diff(df, cluster[i])
    return np.mean(diffs_c)

def bootstrap_cluster_se(df, B=1000):
    diffs = np.zeros(B)
    for i in range(B):
        diffs[i] = cluster_se(df, yob_values)
    return np.std(diffs)

# Set the number of bootstrap samples
B = 1000
# Calculate the bootstrap standard error
bootstrap_se_value = bootstrap_cluster_se(df, B=B)

print(f"Cluster bootstrap standard error of the difference in means is: {bootstrap_se_value:.8f}")


Cluster bootstrap standard error of the difference in means is: 0.00232883


1.e.

In [50]:
# Assuming df has columns: 'yob' and 'y'
results = {}
yob_values = df['yob'].unique()

for yob in yob_values:
    df_c = df[df['yob'] == yob]
    
    if len(df_c) < 2:
        results[yob] = np.nan
        continue
    
    y_norm = (df_c['y'] - df_c['y'].mean()) / df_c['y'].std(ddof=0)
    
    pair_products = [y_norm.iloc[i] * y_norm.iloc[j] for i, j in combinations(range(len(df_c)), 2)]
    
    results[yob] = np.mean(pair_products)

within_cluster_corr = pd.Series(results, name="within_cluster_corr")
print(within_cluster_corr)


In [ ]:
results = {}
for i in range(len(yob_values)):
    df_c = df[df.yob == yob_values[i]]
    y_norm = (df_c['y'] - df_c['y'].mean())/df_c['y'].std()
    pair_products = [y_norm[i] * y_norm[j] for i, j in combinations(range(len(df_c)), 2)]
    results[i] = np.mean(pair_products)
pd.Series(results)


1.f.

In [ ]:
# Assuming df has columns: 'yob' and 'y'
results = {}
yob_values = df['yob'].unique()

for yob in yob_values:
    df_c = df[df['yob'] == yob]
    
    if len(df_c) < 2:
        results[yob] = np.nan
        continue
    
    y_norm = (df_c['educ'] - df_c['educ'].mean()) / df_c['educ'].std(ddof=0)
    
    pair_products = [y_norm.iloc[i] * y_norm.iloc[j] for i, j in combinations(range(len(df_c)), 2)]
    
    results[yob] = np.mean(pair_products)

within_cluster_corr = pd.Series(results, name="within_cluster_corr")
print(within_cluster_corr)


In [ ]:
results = {}
for i in range(len(yob_values)):
    df_c = df[df.yob == yob_values[i]]
    y_norm = (df_c['educ'] - df_c['educ'].mean())/df_c['educ'].std()
    pair_products = [y_norm[i] * y_norm[j] for i, j in combinations(range(len(df_c)), 2)]
    results[i] = np.mean(pair_products)
pd.Series(results)

1.g. Another standard error to report, yes probably. Would want to include a measure of the distribution of cluster size.

2.a. Since this is a random sample of the full census, although there is a correlation between education and log earnings, 